In [ ]:
# Imports

import tensorflow as tf
import pathlib
import os
import pathlib
import IPython
import json
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time


In [ ]:
# downloading the weights oft the base model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (299, 299, 3),
    include_top = True, 
    weights = "imagenet",
    classes=1000,
    classifier_activation="softmax",                  
)

In [ ]:
#saving the downloaded base_model 

saving_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

base_model.save(saving_path)

In [ ]:
#Model

model_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/SavedBaseModel.h5')

base_model = tf.keras.models.load_model(model_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([                                        
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])

# def build_model(hp):

inputs = tf.keras.Input(shape=(299, 299, 3))
# x = augmentation(inputs)
# x = rescaling(x)
x = tf.keras.applications.inception_v3.preprocess_input(inputs)
x = base_model(x, training=False)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# outputs = tf.keras.layers.Dense(1000, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, x)

# learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")

# model.compile(
#     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
#     loss=tf.losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'],
#     run_eagerly = True)                                               
  
#  return model 

# build_model(keras_tuner.HyperParameters())

# tuner = keras_tuner.RandomSearch(
#     hypermodel=build_model,
#     objective="val_accuracy",
#     max_trials=3,
#     executions_per_trial=2,
#     overwrite=True,
#     directory="/content/drive/MyDrive/Transfer",
#     project_name="Tuner",
# )

# tuner.search_space_summary()

# tuner.search(dataset_path, epochs=5, validation_data = dataset_path_val)

# tuner.results_summary()


# model.fit(                                                               # fitting the whole model for non-trainable base
#     dataset_path,
#     epochs=1,
#     validation_data = dataset_path_val)   
    # validation_data = dataset_path_val)
    #verbose = 1)

model.summary()

In [ ]:
# Predicting

train_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/trai/')

val_data_path = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/val/')

data_path_test = pathlib.Path('/home/samer/Desktop/Beedoo/VizWiz/test/')



model.compile(                                                   # compiling the loaded model
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy'])

dictionary = dict()
counter = 0

for file in os.listdir(os.path.join(val_data_path)):
    image_path = os.path.join(val_data_path, file)

    image = tf.keras.utils.load_img(image_path,
                                    target_size=(299,299))
    input_arr = tf.keras.utils.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    # plt.imshow(image)
    # plt.show()
    predictions = model.predict(input_arr)
    pred = np.argmax(predictions, axis=1)
    dictionary[file] = int(pred[0])
    counter +=1
    if counter % 600 == 0:
        time.sleep(5)

print(dictionary)

 
# Writing to sample.json
with open("/home/samer/Desktop/Beedoo/VizWiz/sample_val.json", "a") as outfile:
    json_object = json.dump(dictionary, outfile)

